In [7]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [14]:
Xtr = pd.read_csv('Xtr_Assessed_Value.csv')
Xte = pd.read_csv('Xte_Assessed_Value.csv')
Ytr = pd.read_csv('Ytr.csv')

C:\Users\Tyler\AppData\Local\Temp\ipykernel_17396\4197991358.py:2: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  Xte = pd.read_csv('Xte_Assessed_Value.csv')


In [19]:
# Calculate the 'Ratio' and handle zeros
merged_data = Xtr.merge(Ytr, on='ID')
merged_data['Ratio'] = merged_data['Sale_Amount'] / merged_data['Assessed_Value']
merged_data['Ratio'] = merged_data['Ratio'].replace(0, np.nan)

# Group by 'List_Year' and 'Town' to calculate the mean 'Ratio'
result = merged_data.groupby(['List_Year', 'Town'])[['Ratio']].mean().reset_index()
result['Ratio'] = result['Ratio'].replace([np.inf, -np.inf], np.nan)

# Encode 'Town' using LabelEncoder
label_encoder = LabelEncoder()
result['Town'] = label_encoder.fit_transform(result['Town'])
data = result[['List_Year', 'Town', 'Ratio']]

# Imputation missing Sales_Ratio by using KNNImputer
# Reason: When creating Ratio, there should be missing values since we have missing Assessed_Value.
# https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html
imputer = KNNImputer(n_neighbors=30)
imputed_data = imputer.fit_transform(data)
result['Ratio'] = imputed_data[:, 2]

# Getting Town names back in result column since it was encoded by integer.
Town = merged_data.groupby(['List_Year', 'Town'])[['Ratio']].mean().reset_index()
result['Town'] = Town['Town']

Xtr = Xtr.merge(result[['List_Year', 'Town', 'Ratio']], on=['List_Year', 'Town'], how='left')
mean_sales_ratio = Xtr.groupby(['List_Year', 'Town'])[['Ratio']].mean().reset_index()
Xte = Xte.merge(mean_sales_ratio, on=['List_Year', 'Town'], how='left')

In [20]:
Xtr4 = Xtr[["List_Year", "Assessed_Value", "Town", "Ratio"]]
Xte4 = Xte[["List_Year", "Assessed_Value", "Town", "Ratio"]]

# Calculate feature1Btr
towns = Xtr4["Town"].unique()
feature1Btr = np.empty(len(Xtr4))
for town in towns:
    mask = Xtr4["Town"] == town
    mu = Ytr.loc[mask, "Sale_Amount"].mean()
    feature1Btr[mask] = mu

# Calculate feature1Bte
feature1Bte = np.zeros(len(Xte4))
for town in towns:
    mask = Xtr4["Town"] == town
    mu = Ytr.loc[mask, "Sale_Amount"].mean()
    mask = Xte4["Town"] == town
    feature1Bte[mask] = mu

Xtr4['Assessed_Value'] = np.log1p(Xtr4['Assessed_Value'])
Xte4['Assessed_Value'] = np.log1p(Xte4['Assessed_Value'])

# Add F1B feature
Xtr4["F1B"] = feature1Btr
Xte4["F1B"] = feature1Bte

# Drop the "Town" column
Xtr4.drop("Town", axis=1, inplace=True)
Xte4.drop("Town", axis=1, inplace=True)

C:\Users\Tyler\AppData\Local\Temp\ipykernel_17396\4253608891.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xtr4['Assessed_Value'] = np.log1p(Xtr4['Assessed_Value'])
C:\Users\Tyler\AppData\Local\Temp\ipykernel_17396\4253608891.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xte4['Assessed_Value'] = np.log1p(Xte4['Assessed_Value'])
C:\Users\Tyler\AppData\Local\Temp\ipykernel_17396\4253608891.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [21]:
Xtr4.to_csv('Xtr_Final.csv', index=False)
Xte4.to_csv('Xte_Final.csv', index=False)